# **CORRER EL CÓDIGO HASTA EL PRÓXIMO MENSAJE**

In [62]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import random

In [63]:
season_links = [
    'https://fbref.com/en/comps/12/2023-2024/schedule/2023-2024-La-Liga-Scores-and-Fixtures',
    'https://fbref.com/en/comps/12/2022-2023/schedule/2022-2023-La-Liga-Scores-and-Fixtures',
    'https://fbref.com/en/comps/12/2021-2022/schedule/2021-2022-La-Liga-Scores-and-Fixtures',
    'https://fbref.com/en/comps/12/2020-2021/schedule/2020-2021-La-Liga-Scores-and-Fixtures',
    'https://fbref.com/en/comps/12/2019-2020/schedule/2019-2020-La-Liga-Scores-and-Fixtures',
    'https://fbref.com/en/comps/12/2018-2019/schedule/2018-2019-La-Liga-Scores-and-Fixtures',
    'https://fbref.com/en/comps/12/2017-2018/schedule/2017-2018-La-Liga-Scores-and-Fixtures'
]

In [64]:
def drop_top_column_level(dataframe):
    dataframe.columns = dataframe.columns.droplevel(0)

def merge_players(players, team_id):
    team_df = players[0]
    for i, df in enumerate(players[1:]):
        team_df = pd.merge(
            team_df, df, on='Player', how='outer', 
            suffixes=('', f'_{i+1}')
        )
    team_df = team_df.reset_index(drop=True)
    team_df['team_id'] = team_id
    return team_df

In [ ]:
season_counter = 1
season_df = pd.DataFrame()

# Start the web driver once for all seasons
driver = webdriver.Chrome()

for season in season_links:
    
    # Get links of all the matches in the season
    driver.get(season)
    match_report_links = driver.find_elements(By.XPATH, "//a[text()='Match Report']")
    links = [link.get_attribute("href") for link in match_report_links]

    # Collect all match data for the season
    match_data_list = []
    
    for counter, link in enumerate(links, start=1):
        
        time.sleep(random.uniform(5, 10))  # Optional, if 429 errors are a concern
        
        tables = pd.read_html(link, attrs={'class': 'stats_table'})
        
        # Drop the top column level for all tables
        for table in tables:
            drop_top_column_level(table)
        
        # Process home and away player data
        home_players = tables[0:7]
        away_players = tables[7:14]
        
        home_df = merge_players(home_players, 'H')[1:]
        away_df = merge_players(away_players, 'A')[1:]
        
        match_players = pd.concat([home_df, away_df]).reset_index(drop=True)
        match_players['match_id'] = counter
        match_data_list.append(match_players)

        print(f"Completed match {counter} of season {season_counter}")

    # Combine all matches in the season into a single DataFrame
    all_df = pd.concat(match_data_list).reset_index(drop=True)

    # Get season fixtures
    calendar = pd.read_html(season)[0].dropna(how='all')
    calendar['match_id'] = range(1, len(calendar) + 1)
    fixtures = calendar[['match_id', 'Home', 'Away']]
    
    # Melt fixtures to match home/away teams with match_id
    df_melted = fixtures.melt(id_vars=['match_id'], value_vars=['Home', 'Away'], 
                              var_name='team_id', value_name='team_name')
    df_melted['team_id'] = df_melted['team_id'].replace({'Home': 'H', 'Away': 'A'})

    # Merge player data with team names from fixtures
    complete_df = all_df.merge(df_melted[['match_id', 'team_id', 'team_name']], 
                               on=['match_id', 'team_id'], how='left')
    
    complete_df['season_id'] = season_counter

    complete_df.to_csv(f'laliga_season_id_{2024 - season_counter}_{2025 - season_counter}')

    season_counter += 1
    
    # Append complete_df for the season to the season_df
    season_df = pd.concat([season_df, complete_df], ignore_index=True)

    
    print(f"Completed season {season_counter - 1}")

# Close the web driver once at the end
driver.quit()

Completed match 1 of season 1
Completed match 2 of season 1
Completed match 3 of season 1
Completed match 4 of season 1
Completed match 5 of season 1
Completed match 6 of season 1
Completed match 7 of season 1
Completed match 8 of season 1
Completed match 9 of season 1
Completed match 10 of season 1
Completed match 11 of season 1
Completed match 12 of season 1
Completed match 13 of season 1
Completed match 14 of season 1
Completed match 15 of season 1
Completed match 16 of season 1
Completed match 17 of season 1
Completed match 18 of season 1
Completed match 19 of season 1
Completed match 20 of season 1
Completed match 21 of season 1
Completed match 22 of season 1
Completed match 23 of season 1
Completed match 24 of season 1
Completed match 25 of season 1
Completed match 26 of season 1
Completed match 27 of season 1
Completed match 28 of season 1
Completed match 29 of season 1
Completed match 30 of season 1
Completed match 31 of season 1
Completed match 32 of season 1
Completed match 3

KeyboardInterrupt: 

In [ ]:
season_df.to_csv('tables/laliga_all_seasons.csv')

# **HASTA AQUÍ, LO SIGUIENTE ES SOLO DE PRUEBA :)**

In [48]:
driver = webdriver.Chrome()
driver.get("https://fbref.com/en/comps/12/2023-2024/schedule/2023-2024-La-Liga-Scores-and-Fixtures")

match_report_links = driver.find_elements(By.XPATH, "//a[text()='Match Report']")
links = [link.get_attribute("href") for link in match_report_links]

driver.quit()

In [50]:
all_df = pd.DataFrame()
counter = 1

for i in links:
    time.sleep(random.uniform(5, 10))

    tables = pd.read_html(i, attrs={'class': 'stats_table'})

    for table in tables:
        drop_top_column_level(table)

    home_players = tables[0:7]
    away_players = tables[7:14]

    home_df = merge_players(home_players, 'H')[1:]
    away_df = merge_players(away_players, 'A')[1:]

    match_players = pd.concat([home_df, away_df]).reset_index(drop=True)
    match_players['match_id'] = counter

    counter += 1

    if all_df.empty:
        all_df = match_players
    else:
        all_df = pd.concat([all_df, match_players]).reset_index(drop=True)
    
    print(counter-1)

1


KeyboardInterrupt: 

In [51]:
all_df

,Player,#,Nation,Pos,Age,Min,Gls,Ast,PK,PKatt,...,Att_6,Launch%,AvgLen,Opp,Stp,Stp%,#OPA,AvgDist,team_id,match_id
0,Adri Embarba,10.0,es ESP,"LW,LM",31-096,90,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,H,1
1,Alejandro Pozo Pozo,17.0,es ESP,RB,24-170,65,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,H,1
2,Diego Mariño,1.0,es ESP,GK,33-094,90,0,0,0,0,...,2.0,0.0,16.5,6.0,0.0,0.0,2.0,22.7,H,1
3,Dion Lopy,6.0,sn SEN,CM,21-190,13,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,H,1
4,Gonzalo Melero,11.0,es ESP,CM,29-221,14,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,H,1
5,Iddrisu Baba,4.0,gh GHA,"DM,CM",27-201,76,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,H,1
6,Largie Ramazani,7.0,be BEL,"AM,RM",22-165,82,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,H,1
7,Lucas Robertone,5.0,ar ARG,"DM,CM",26-146,77,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,H,1
8,Luis Javier Suárez,9.0,co COL,FW,25-252,90,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,H,1
9,Lázaro,14.0,br BRA,RW,21-152,45,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,H,1


In [52]:
all_df = pd.read_csv('tables/laliga_23_24.csv')
all_df

,Player,#,Nation,Pos,Age,Min,Gls,Ast,PK,PKatt,...,TklW_5,PKwon,PKcon,OG,Recov,Won,Lost_5,Won%,team_id,match_id
0,Adri Embarba,10.0,es ESP,"LW,LM",31-096,90,0,0,0,0,...,2,0,0,0,5,1,2,33.3,H,1
1,Alejandro Pozo Pozo,17.0,es ESP,RB,24-170,65,0,0,0,0,...,0,0,0,0,1,1,0,100.0,H,1
2,Diego Mariño,1.0,es ESP,GK,33-094,90,0,0,0,0,...,0,0,0,0,1,0,0,NaN,H,1
3,Dion Lopy,6.0,sn SEN,CM,21-190,13,0,0,0,0,...,0,0,0,0,1,0,0,NaN,H,1
4,Gonzalo Melero,11.0,es ESP,CM,29-221,14,0,0,0,0,...,0,0,0,0,0,0,0,NaN,H,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11883,Pau Cubarsí,33.0,es ESP,CB,17-125,90,0,0,0,0,...,0,0,0,0,3,0,1,0.0,A,380
11884,Pedri,8.0,es ESP,RM,21-183,90,0,0,0,0,...,0,0,0,0,5,0,1,0.0,A,380
11885,Robert Lewandowski,9.0,pl POL,FW,35-279,61,1,0,0,0,...,0,0,0,0,4,0,0,NaN,A,380
11886,Vitor Roque,19.0,br BRA,FW,19-088,16,0,0,0,0,...,0,0,0,0,1,0,0,NaN,A,380


In [36]:
calendar = pd.read_html('https://fbref.com/en/comps/12/2023-2024/schedule/2023-2024-La-Liga-Scores-and-Fixtures')[0].dropna(how='all')

In [53]:
calendar['match_id'] = range(1, len(calendar) + 1)

In [54]:
fixtures = calendar.copy()

In [55]:
fixtures = fixtures[['match_id', 'Home', 'Away']]

df_melted = fixtures.melt(id_vars=['match_id'], value_vars=['Home', 'Away'], 
                    var_name='team_id', value_name='team_name')

df_melted['team_id'] = df_melted['team_id'].replace({'Home': 'H', 'Away': 'A'})

In [56]:
df_melted

,match_id,team_id,team_name
0,1,H,Almería
1,2,H,Sevilla
2,3,H,Real Sociedad
3,4,H,Las Palmas
4,5,H,Athletic Club
...,...,...,...
755,376,A,Betis
756,377,A,Mallorca
757,378,A,Alavés
758,379,A,Valencia


In [60]:
complete_df = all_df.merge(df_melted[['match_id', 'team_id', 'team_name']], on=['match_id', 'team_id'], how='left')

In [61]:
complete_df

,Player,#,Nation,Pos,Age,Min,Gls,Ast,PK,PKatt,...,PKwon,PKcon,OG,Recov,Won,Lost_5,Won%,team_id,match_id,team_name
0,Adri Embarba,10.0,es ESP,"LW,LM",31-096,90,0,0,0,0,...,0,0,0,5,1,2,33.3,H,1,Almería
1,Alejandro Pozo Pozo,17.0,es ESP,RB,24-170,65,0,0,0,0,...,0,0,0,1,1,0,100.0,H,1,Almería
2,Diego Mariño,1.0,es ESP,GK,33-094,90,0,0,0,0,...,0,0,0,1,0,0,NaN,H,1,Almería
3,Dion Lopy,6.0,sn SEN,CM,21-190,13,0,0,0,0,...,0,0,0,1,0,0,NaN,H,1,Almería
4,Gonzalo Melero,11.0,es ESP,CM,29-221,14,0,0,0,0,...,0,0,0,0,0,0,NaN,H,1,Almería
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11883,Pau Cubarsí,33.0,es ESP,CB,17-125,90,0,0,0,0,...,0,0,0,3,0,1,0.0,A,380,Barcelona
11884,Pedri,8.0,es ESP,RM,21-183,90,0,0,0,0,...,0,0,0,5,0,1,0.0,A,380,Barcelona
11885,Robert Lewandowski,9.0,pl POL,FW,35-279,61,1,0,0,0,...,0,0,0,4,0,0,NaN,A,380,Barcelona
11886,Vitor Roque,19.0,br BRA,FW,19-088,16,0,0,0,0,...,0,0,0,1,0,0,NaN,A,380,Barcelona
